In [1]:
import polars as pl
from scipy.sparse import coo_array, diags
from sksparse.cholmod import cholesky

from benchmarks.utils import mock_snakemake

DF_CUTOFF = 1e-5

if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("compute_pdf")

lines = pl.read_parquet(snakemake.input.lines)
buses = pl.concat([lines["from_bus"], lines["to_bus"]]).unique()
L = len(lines)
N = len(buses)
lines.sort("from_bus")

line_id,from_bus,to_bus,line_rating,reactance
u32,i64,i64,f64,f64
1125,1,2410,1.75,0.000219
1432,3,3975,0.8,0.052057
2624,3,3972,0.64,0.027643
3657,3,3976,0.3,0.018766
7884,3,3971,0.64,0.027293
…,…,…,…,…
1287,8854,8855,0.940328,0.115871
2152,8860,8862,5.29,0.020603
1246,8861,8862,3.13,0.034818


In [2]:
# Step 1: Renumber buses such that they go from 1 to N contiguously.
bus_mapping = (
    buses.sort().to_frame().with_row_index(name="new_bus").rename({"from_bus": "bus"})
)
lines = (
    lines.join(bus_mapping.rename({"bus": "from_bus"}), on="from_bus")
    .drop("from_bus")
    .rename({"new_bus": "from_bus"})
    .join(bus_mapping.rename({"bus": "to_bus"}), on="to_bus")
    .drop("to_bus")
    .rename({"new_bus": "to_bus"})
)
lines.sort("from_bus")

line_id,line_rating,reactance,from_bus,to_bus
u32,f64,f64,u32,u32
1125,1.75,0.000219,0,1982
1432,0.8,0.052057,1,2962
2624,0.64,0.027643,1,2960
3657,0.3,0.018766,1,2963
7884,0.64,0.027293,1,2959
…,…,…,…,…
1287,0.940328,0.115871,6461,6462
2152,5.29,0.020603,6464,6466
1246,3.13,0.034818,6465,6466


In [3]:
# Step 2: Form an admittance matrix.
lines = lines.with_columns(suscep=1 / lines["reactance"])
off_diagonal = pl.concat(
    [
        lines.select(
            pl.col("from_bus").alias("i"),
            pl.col("to_bus").alias("j"),
            -pl.col("suscep"),
        ),
        lines.select(
            pl.col("to_bus").alias("i"),
            pl.col("from_bus").alias("j"),
            -pl.col("suscep"),
        ),
    ]
)
# TODO double check
diagonal = (
    pl.concat(
        [
            lines.select("suscep", i=pl.col("from_bus")),
            lines.select("suscep", i=pl.col("to_bus")),
        ]
    )
    .group_by("i")
    .agg(pl.sum("suscep"))
    .select("i", pl.col("i").alias("j"), "suscep")
)
assert len(diagonal) == N, "Diagonal entries should match the number of buses."
admittance_matrix_df = pl.concat([off_diagonal, diagonal])
admittance_matrix_df

i,j,suscep
u32,u32,f64
1150,2675,-1494.984952
5073,5890,-1.664108
5364,6016,-69.61868
2897,2901,-104.953277
5376,5379,-109.510177
…,…,…
5502,5502,163.231729
3141,3141,138.551671
5505,5505,946.742598


In [4]:
# Step 3: Drop last row and column (slack bus).
admittance_matrix_df = admittance_matrix_df.filter(
    pl.col("i") < (N - 1), pl.col("j") < (N - 1)
)
admittance_matrix = coo_array(
    (
        admittance_matrix_df["suscep"].to_numpy(),
        (
            admittance_matrix_df["i"].cast(pl.Float64).to_numpy(),
            admittance_matrix_df["j"].cast(pl.Float64).to_numpy(),
        ),
    ),
    shape=(N - 1, N - 1),
)
admittance_matrix

<COOrdinate sparse array of dtype 'float64'
	with 22714 stored elements and shape (6472, 6472)>

In [5]:
# Step 4: Inverse the admittance matrix to get voltage angles.
# We use the sparse Cholesky factorization since, based on my tests, this is much faster than using
# scipy.linalg.inv, scipy.sparse.linalg.inv, scipy.linag.pinv, scipy.sparse.linalg.spsolve
factor = cholesky(admittance_matrix.tocsc())
voltage_angles = factor.inv()
voltage_angles

/tmp/ipykernel_1943993/646270145.py:5: CholmodTypeConversionWarning: array contains 32 bit integers; but 64 bit integers are needed; slowing down due to converting
  voltage_angles = factor.inv()


<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 41886784 stored elements and shape (6472, 6472)>

In [6]:
adjacency_matrix_T = pl.concat(
    [
        lines.select(
            i=pl.col("line_id") - 1,  # Adjusting for zero-based indexing
            j=pl.col("from_bus"),
            val=pl.lit(1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus,
        lines.select(
            i=pl.col("line_id") - 1,  # Adjusting for zero-based indexing
            j=pl.col("to_bus"),
            val=pl.lit(-1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus
    ]
)
adjacency_matrix_T = coo_array(
    (
        adjacency_matrix_T["val"].to_numpy(),
        (adjacency_matrix_T["i"].to_numpy(), adjacency_matrix_T["j"].to_numpy()),
    ),
    shape=(len(lines), N - 1),
)
susceptance_matrix = diags(lines.sort("line_id")["suscep"].to_numpy())
power_flow = susceptance_matrix @ coo_array(adjacency_matrix_T) @ voltage_angles
power_flow = power_flow.tocoo()
power_flow_df = pl.DataFrame(
    {
        "injection": power_flow.col,
        "line": power_flow.row + 1,
        "factor": power_flow.data,
    }
)
power_flow_df

injection,line,factor
i32,i32,f64
6471,1,0.003544
6470,1,-0.000056
6469,1,-0.000056
6468,1,-0.000057
6467,1,-0.000057
…,…,…
4,8124,-3.9080e-14
3,8124,-3.9968e-14
2,8124,-3.9080e-14


In [7]:
if DF_CUTOFF is not None:
    power_flow_df = power_flow_df.filter(pl.col("factor").abs() > DF_CUTOFF)
power_flow_df

injection,line,factor
i32,i32,f64
6471,1,0.003544
6470,1,-0.000056
6469,1,-0.000056
6468,1,-0.000057
6467,1,-0.000057
…,…,…
3248,8124,1.0
1297,8124,1.0
1296,8124,1.0


In [8]:
# Unmap buses to original IDs.
power_flow_df = (
    power_flow_df.join(bus_mapping.rename({"new_bus": "injection"}), on="injection")
    .drop("injection")
    .rename({"bus": "injection"})
)

In [9]:
power_flow_df.write_parquet(snakemake.output[0])
snakemake.output[0]

'/data/home/machstg/pyoframe/benchmarks/input_data/energy_planning/power_distribution_factors.parquet'